# Data Extraction

In [ ]:
# Normal imports
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#Specific imports
import geopandas
from geopandas import GeoDataFrame
from shapely import wkt
from shapely.geometry import Point
from sklearn.model_selection import train_test_split


#Internal imports
from src.data_extraction.data_extraction import extract_initial_data
from src.constants import (
    NEW_COLUMNS_NAMES,
    REMOVE_COLUMNS_BY_INPUT,
    REMOVE_COLUMNS_BY_CORRELATIONS,
)
from src.preprocessing.preprocessing_utils import (
    remove_duplicated_anuncios_id,
    find_single_value_columns,
    treatment_missing_values,
    correlation_values,
    feature_engineering,
    detect_outliers,
    hist_plot_outliers
)

# Settings
warnings.filterwarnings("ignore")
%matplotlib inline
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Extract all dataset available, provided by idealista

In [ ]:
df, df_ine, df_osm, df_pois, df_polygons = extract_initial_data(
    root_dir="input_data"
)

In [ ]:
# Change columns names to friendly ones
df = df.drop(columns=["ADTYPOLOGY", "ADOPERATION"])
df.columns = NEW_COLUMNS_NAMES

Division train test

In [ ]:
X = df.drop(columns=['precio']) 
y = df['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

## Polygons and INE censal polygons

#### Polygons

In [ ]:
def get_info_from_polygons_and_ine(df_polygons, df_ine, df):
    # Convert WKT strings to Shapely geometries and create a GeoDataFrame
    df_polygons['geometry'] = df_polygons['WKT'].apply(wkt.loads)
    gdf_polygons = geopandas.GeoDataFrame(df_polygons['geometry'], crs='epsg:4326')

    # Add additional columns to the GeoDataFrame
    gdf_polygons['barrio_id'] = df_polygons['LOCATIONID']
    gdf_polygons['barrio'] = df_polygons['LOCATIONNAME']

    # Create Point geometries using longitude and latitude coordinates from df_train
    geometry = [Point(xy) for xy in zip(df.longitud, df.latitud)]

    # Create a GeoDataFrame gdf_ads with df_prices data and geometry column
    gdf_train_train = GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

    # Apply a logarithmic scale transformation to the 'precio' column in gdf_ads
    gdf_train_train['precio_logaritmico'] = np.log(gdf_train_train['precio'])

    # Convert WKT strings to Shapely geometries and create a GeoDataFrame for census polygons
    df_ine['geometry'] = df_ine['WKT'].apply(wkt.loads)
    gdf_polygons_census = geopandas.GeoDataFrame(df_ine['geometry'], crs='epsg:4326')

    # Add additional column 'CUSEC' to the GeoDataFrame representing census polygons
    gdf_polygons_census['cusec'] = df_ine['CUSEC']

    # Add the census codes (CUSEC)
    gdf_train_train = geopandas.sjoin(gdf_train_train, gdf_polygons_census, how="inner")

    # Drop index_right 
    gdf_train_train = gdf_train_train.drop(columns=['index_right'])

    # Now add the idealista zones (LOCATIONID, LOCATIONNAME)
    gdf_train_train = geopandas.sjoin(gdf_train_train, gdf_polygons, how="inner")

    # Drop index_right 
    gdf_train_train = gdf_train_train.drop(columns=['index_right'])

    return gdf_train_train


In [ ]:
df_train = get_info_from_polygons_and_ine(df_polygons=df_polygons, df_ine=df_ine, df=df_train)
df_test = get_info_from_polygons_and_ine(df_polygons=df_polygons, df_ine=df_ine, df=df_test)

#### INE Censal Polygons

## ASSETS - Basis

### General

In [ ]:
# See general statistics of df
description_df = df_train.describe(percentiles=[0.995, 0.97]).transpose().style.format("{:.2f}")
description_df

In [ ]:
# Remove duplicated anuncios_id
df_train = remove_duplicated_anuncios_id(df_assets=df_train, criteria="last")

In [ ]:
# Remove columns by input
df_train = df_train.drop(columns=REMOVE_COLUMNS_BY_INPUT)
df_test = df_test.drop(columns=REMOVE_COLUMNS_BY_INPUT)
print('Removed columns:', REMOVE_COLUMNS_BY_INPUT)

In [ ]:
# Remove columns that only have one different value
remove_unique_value_columns = find_single_value_columns(df=df_train)
df_train = df_train.drop(columns=remove_unique_value_columns)
df_test = df_test.drop(columns=remove_unique_value_columns)

### Missing Values

In [ ]:
# Missing values
df_train = treatment_missing_values(df=df_train)

In [ ]:
#Ano Construccion
antiguidade_ano_construccion=2018-df_train['ano_construccion']
print('Min antiguidade ano construccion', antiguidade_ano_construccion.min())
print('Max antiguidade ano construccion', antiguidade_ano_construccion.max())
antiguidade_cat_ano_construccion=2018-df_train['cat_ano_construccion']
print('Min antiguidade cat ano construccion', antiguidade_cat_ano_construccion.min())
print('Max antiguidade cat ano construccion', antiguidade_cat_ano_construccion.max())

### Data Treatment and Feature Engineering

In [ ]:
# Mean price by barrio
df_metrics_barrios= df_train.groupby(['barrio']).agg({'precio':['median', 'mean', 'std'], 
                                   'precio_unitario_m2':['median', 'mean', 'std']}).reset_index()

df_metrics_barrios.columns = ['barrio', 
                         'precio_median_barrio', 'precio_mean_barrio', 'precio_std_barrio', 
                         'precio_unitario_m2_median_barrio', 'precio_unitario_m2_mean_barrio', 'precio_unitario_m2_std_barrio']

In [ ]:
# Add mean price by barrio to dataset
df_train=df_train.merge(df_metrics_barrios[['barrio', 'precio_mean_barrio', 'precio_unitario_m2_mean_barrio']], on=['barrio'], how="inner")
df_test=df_test.merge(df_metrics_barrios[['barrio', 'precio_mean_barrio', 'precio_unitario_m2_mean_barrio']], on=['barrio'], how="inner")


In [ ]:
# Feature Engineering: add new variables
(
    add_columns,
    remove_columns_by_creating_new_variables,
    df_train,
) = feature_engineering(df=df_train)

print("Columns added:", add_columns)
print('Columns removed by adding new:', remove_columns_by_creating_new_variables)

(
    add_columns,
    remove_columns_by_creating_new_variables,
    df_test,
) = feature_engineering(df=df_test)

Correlations

In [ ]:
# Correlation values
correlation_matrix, correlated_variables = correlation_values(df=df_train, threshold=0.8)
# Remove columns by high correlations
df_train = df_train.drop(columns=REMOVE_COLUMNS_BY_CORRELATIONS)

df_test = df_test.drop(columns=REMOVE_COLUMNS_BY_CORRELATIONS)

Outliers

In [ ]:
nr_row_before=df_train.shape[0]

variables_most_correlated_w_target=['n_banos','n_habitaciones', 'area_construida', 'distancia_castellana']

percentile_995_values = {}

for var in variables_most_correlated_w_target:
    percentile_995_values[var] = df_train[var].quantile(0.995)

print(percentile_995_values)

for var in variables_most_correlated_w_target:
    hist_plot_outliers(df=df_train[df_train[var] > percentile_995_values[var]],name_variable = var)

condition_to_exclude_outliers = ((df_train['n_banos']>percentile_995_values['n_banos']) 
                                 | (df_train['n_habitaciones']>percentile_995_values['n_habitaciones']) 
                                 | (df_train['area_construida']>percentile_995_values['area_construida']) 
                                 | (df_train['distancia_castellana']>percentile_995_values['distancia_castellana']))

df_train=df_train[~condition_to_exclude_outliers]

nr_row_after=df_train.shape[0]

print('Percentage of rows deleted: ', 1-nr_row_after/nr_row_before)
    

In [ ]:
#FIXME
# Analyse target outliers: inter quartis
outliers = detect_outliers(df=df_train, column_name='precio', threshold=10)
outliers.shape

N_Banos: Impute values when n_banos = 0 --> train

In [ ]:
# Filter rows where n_banos > 0
filtered_df = df_train[df_train['n_banos'] > 0]

# Calculate number of bathrooms per square meter
filtered_df['n_banos_m2'] = filtered_df['n_banos'] / filtered_df['area_construida']

# Calculate mean number of bathrooms per square meter
mean_bathrooms_per_sqm = filtered_df['n_banos_m2'].median()

# Impute number of bathrooms for rows where n_banos == 0: mean number of bath by m^2 * m^2, rounded, and minimum 1
df_train['n_banos_m2'] = (np.maximum(mean_bathrooms_per_sqm * df_train['area_construida'],1)).round().astype(int)

# Validate if it is a good way to values, calculating mape comparing with the real n_banos
df_train_aux = df_train[df_train['n_banos']>0]
absolute_percentage_errors = np.abs((df_train_aux['n_banos'] - df_train_aux['n_banos_m2']) / df_train_aux['n_banos'])
mape = np.mean(absolute_percentage_errors) * 100
print("MAPE", mape)

# Delete filtered_df, df_train_aux as they are only auxiliar 
del filtered_df, df_train_aux

# # Assign the imputed value to n_banos where n_banos == 0. All houses with 0 bathrooms are houses to renovate
df_train.loc[df_train['n_banos'] == 0, 'n_banos'] = df_train.loc[df_train['n_banos'] == 0, 'n_banos_m2']

# # # Drop the n_banos_m2 column as it is no longer needed
df_train = df_train.drop(columns=['n_banos_m2'])

# Exportar df_train_util

In [155]:
df_train.to_csv(r'output_data\df_train_util.csv', index=False)

In [ ]:
df_test.fillna(0).to_csv(r'output_data\df_test_util.csv', index=False)

In [ ]:
# Compare data types for common columns
for col in df_train.columns.intersection(df_test.columns):
    dtype_df1 = df_train[col].dtype
    dtype_df2 = df_test[col].dtype
    if dtype_df1 != dtype_df2:
        print(f"Data type of column '{col}' differs between DataFrames:")
        print(f"- DataFrame 1: {dtype_df1}")
        print(f"- DataFrame 2: {dtype_df2}")


In [ ]:
df_test[df_test.latitud > 40]